In [10]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import joblib

# Load Dataset
df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")

# Drop non-informative ID column
df.drop('Person ID', axis=1, inplace=True)

# Fix fillna warning
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')

# Split Blood Pressure into two columns
df['systolic_bp'] = df['Blood Pressure'].apply(lambda x: int(x.split('/')[0]))
df['diastolic_bp'] = df['Blood Pressure'].apply(lambda x: int(x.split('/')[1]))
df.drop('Blood Pressure', axis=1, inplace=True)

# Clean BMI category
df['BMI Category'] = df['BMI Category'].replace('Normal Weight', 'Normal')

# Encode categorical variables
label_columns = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']
le = LabelEncoder()
for col in label_columns:
    df[col] = le.fit_transform(df[col])

# Balance classes
df_class_0 = df[df['Sleep Disorder'] == 0]
df_class_1 = df[df['Sleep Disorder'] == 1]
df_class_2 = df[df['Sleep Disorder'] == 2]

max_size = df_class_1.shape[0]
df_class_0_up = resample(df_class_0, replace=True, n_samples=max_size, random_state=42)
df_class_2_up = resample(df_class_2, replace=True, n_samples=max_size, random_state=42)

df_balanced = pd.concat([df_class_1, df_class_0_up, df_class_2_up])
df_balanced = df_balanced.sample(frac=1, random_state=42)

# Train/Test Split
X = df_balanced.drop('Sleep Disorder', axis=1)
y = df_balanced['Sleep Disorder']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train Model
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# Save Model and Files
joblib.dump(rfc, "sleep_disorder_model.pkl")
joblib.dump(X.columns.tolist(), "feature_columns.pkl")
joblib.dump(le, "label_encoder.pkl")

print("✅ Model trained and saved successfully.")


✅ Model trained and saved successfully.
